In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import datetime
import plotly
import sys
from google.cloud import bigquery

sys.path.append("../src")

In [2]:
from etl_processor import ETLProcessor
from connectors import AlphaAdvantage, FMPClient
from bigquery_connector import BigQueryConnector
from web_scrapper import WebScrapper

In [3]:
alpha_api_url = os.getenv("ALPHA_API_URL")
alpha_api_key = os.getenv("ALPHA_API_KEY")

fmp_api_url = os.getenv("FMP_API_URL")
fmp_api_key = os.getenv("FMP_API_KEY")

bigquery_cred = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
project_id = os.getenv("GCP_PROJECT_ID")
table_id = os.getenv("RAW_DATA_TABLE")

In [4]:
data_path = '../src/s&p500_ticker.csv'
data = pd.read_csv(data_path, index_col=0) # ignore col 0
ticker_list = data['Ticker'].tolist()

## ETL TESTER

In [5]:
alpha_client = AlphaAdvantage(alpha_api_url, alpha_api_key)
fmp_client = FMPClient(fmp_api_url, fmp_api_key)
bigquery_client = BigQueryConnector(project_id, bigquery_cred)
# web_scrap = WebScrapper()

In [6]:
# res = requests.get(f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol=NVDA&timeseries=365&apikey={fmp_api_key}")
# data = res.json()
# print(data)

In [7]:
etl_process = ETLProcessor(alpha_client, fmp_client, bigquery_client)
data = etl_process.fmp_extract(ticker_list)

2025-12-11 21:26:03,296 - INFO - Extracting ticker symbol: NVDA


In [8]:
print(data)

[[{'symbol': 'NVDA', 'date': '2025-12-11', 'open': 180.275, 'high': 181.31, 'low': 176.62, 'close': 180.96, 'volume': 157846534, 'change': 0.685, 'changePercent': 0.37997504, 'vwap': 179.63}, {'symbol': 'NVDA', 'date': '2025-12-10', 'open': 184.97, 'high': 185.48, 'low': 182.04, 'close': 183.78, 'volume': 162785400, 'change': -1.19, 'changePercent': -0.64335, 'vwap': 184.0675}, {'symbol': 'NVDA', 'date': '2025-12-09', 'open': 185.56, 'high': 185.72, 'low': 183.32, 'close': 184.97, 'volume': 144719705, 'change': -0.59, 'changePercent': -0.31796, 'vwap': 184.8925}, {'symbol': 'NVDA', 'date': '2025-12-08', 'open': 182.64, 'high': 188, 'low': 182.4, 'close': 185.55, 'volume': 204378106, 'change': 2.91, 'changePercent': 1.59, 'vwap': 184.6475}, {'symbol': 'NVDA', 'date': '2025-12-05', 'open': 183.89, 'high': 184.66, 'low': 180.91, 'close': 182.41, 'volume': 143971100, 'change': -1.48, 'changePercent': -0.80483, 'vwap': 182.9675}, {'symbol': 'NVDA', 'date': '2025-12-04', 'open': 181.62, 'hig

In [9]:
df = etl_process.transform(data)

2025-12-11 21:26:03,860 - INFO - Transformed 1255 records


In [ ]:
print(type(df))
print(type(df['date']))

<class 'NoneType'>


TypeError: 'NoneType' object is not subscriptable

## BigQuery TESTER

In [ ]:
client = BigQueryConnector(project_id, bigquery_cred)

In [ ]:
print(client)

In [ ]:
client.load_dataframe(df, table_id)